In [29]:
#Importação da biblioteca numpy que será essencial para a vetorização dos exemplos

In [2]:
import numpy as np

In [4]:
def backSubstitution(n,U,d):
    x = d/U.diagonal()
    
    for i in range(n-1, -1,-1):
        
        Soma = 0
        
        for j in range(i+1,n,1):
            Soma = Soma + U[i][j] * x[j]
        x[i] = (d[i] - Soma) / U[i][i]
    
    return x

In [5]:
def forwardSubstitution(n,L,c):
    x = np.zeros((3,), dtype='f') #creat output array
    x[0] = c[0]/L[0][0] #initialize array 

    j = 0
    for i in range(1, n, 1):
        soma = 0
        for j in range(0, i-1, 1):
            soma = soma + L[i][j] * x[j]
        
        x[i] = (c[i] - soma) / L[i][j]

In [22]:
def gaussElimination(A,b,n):
    
    for k in range(0,n-1,1):
        for i in range(k+1, n,1):
            m = (-1) * A[i][k] / A[k][k]
            for j in range (k, n, 1):
                A[i][j] = A[i][j] + m * A[k][j]
            b[i] = b[i] + m * b[k] 
    
    
    return backSubstitution(n, A, b)

In [31]:
def gaussPivot(A,b,n):
    
    for k in range(0,n-1,1):
        
        maior = abs(A[k][k])
        for i in range(k+1, n, 1):
            if abs(A[i][k]) > maior:
                maior = abs(A[i][k])
                r = i
            else: 
                r = k
        
        for i in range(0, n, 1):
            v = A[k][i]
            A[k][i] = A[r][i]
            A[r][i] = v 
        
        v = b[k]
        b[k] = b[r]
        b[r] = v
        
        for i in range(k+1, n, 1):
            
            m = (-1) * A[i][k] / A[k][k]
            for j in range (k, n, 1):
                A[i][j] = A[i][j] + m * A[k][j] 
                
                
            b[i] = b[i] + m * b[k]
                  
    return backSubstitution(n, A, b)

In [27]:
n = 3  
A = np.array([[1, -3, 2], [-2, 8, -1], [4, -6, 5]], dtype='f')
b = np.array([11, -15, 29], dtype='f')

In [28]:
resultado = gaussElimination(A,b,n)
print(resultado)


[ 2. -1.  3.]


In [25]:
resultado = gaussPivot(A,b,n)
print(resultado)

[ 2. -1.  3.]


In [30]:
resultado = gaussLU(A,b,n)
print(resultado)

[ 1.99999928 -1.          3.00000009]
